In [1]:
# import libraries
import re
import pandas as pd
import recordlinkage # manages the entire record linkage process

from recordlinkage.index import Full

In [39]:
path = "/Users/zhangyicheng/Library/CloudStorage/Dropbox/My Mac (张苡铖’s MacBook Pro (2))/Desktop/MACS122/Project/"
authors_df = pd.read_csv(path +"affiliation.csv")
authors_df.head()

,affiliationid,name,email
0,au.aalto.fi,Aalto University,aalto.fi
1,kriot.kth.se,KTH Royal Institute of Technology,kth.se
2,uocla.ucla.edu,"University of California, Los Angeles",ucla.edu
3,aei.aei.org,American Enterprise Institute,aei.org
4,miot.mit.edu,Massachusetts Institute of Technology,mit.edu


In [40]:
authors_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1392 entries, 0 to 1391
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   affiliationid  1392 non-null   object
 1   name           1392 non-null   object
 2   email          1392 non-null   object
dtypes: object(3)
memory usage: 32.8+ KB


In [41]:
authors_df["institution"] = authors_df.name

In [42]:
for i, row in authors_df.iterrows():
    ls = row.loc["name"].split(",")
    if len(ls) > 1:
        ele = ls[1]
    else:
        ele = ls[0]
    row.loc['institution'] = ele

In [43]:
authors_df

,affiliationid,name,email,institution
0,au.aalto.fi,Aalto University,aalto.fi,Aalto University
1,kriot.kth.se,KTH Royal Institute of Technology,kth.se,KTH Royal Institute of Technology
2,uocla.ucla.edu,"University of California, Los Angeles",ucla.edu,Los Angeles
3,aei.aei.org,American Enterprise Institute,aei.org,American Enterprise Institute
4,miot.mit.edu,Massachusetts Institute of Technology,mit.edu,Massachusetts Institute of Technology
...,...,...,...,...
1387,smsu.semo.edu,Southeast Missouri State University,semo.edu,Southeast Missouri State University
1388,sc.statcan.gc.ca,Statistics Canada,statcan.gc.ca,Statistics Canada
1389,uofh.ufh.ac.za,University of Fort Hare,ufh.ac.za,University of Fort Hare
1390,uoi.unilorin.edu.ng,University of Ilorin,unilorin.edu.ng,University of Ilorin


In [44]:
path = "/Users/zhangyicheng/Library/CloudStorage/Dropbox/My Mac (张苡铖’s MacBook Pro (2))/Desktop/MACS122/Project/code/"
university_df = pd.read_csv(path +"university_ranking.csv")
university_df = university_df.rename(columns={"rank": "Rank"})
university_df

,Unnamed: 0,University,Rank
0,0,London School of Economics,1
1,1,World Bank Group,2
2,2,National Bureau of Economic Research,3
3,3,International Monetary Fund,4
4,4,Harvard University,5
...,...,...,...
369,541,Bilkent Üniversitesi,542
370,543,Royal Holloway,544
371,544,briq Institute on Behavior and Inequality,545
372,546,Universitá della Svizzera Italiana,547


In [45]:
# To make the comparison more sensitive, strip "university" and "of" from the string
authors_df["institution"] = authors_df["institution"].apply(lambda x: x.lower()).apply(lambda x: x.strip())
authors_df["institution"] = authors_df["institution"].apply(lambda x: x.replace("university",""))
authors_df["institution"] = authors_df["institution"].apply(lambda x: x.replace("of",""))
authors_df

,affiliationid,name,email,institution
0,au.aalto.fi,Aalto University,aalto.fi,aalto
1,kriot.kth.se,KTH Royal Institute of Technology,kth.se,kth royal institute technology
2,uocla.ucla.edu,"University of California, Los Angeles",ucla.edu,los angeles
3,aei.aei.org,American Enterprise Institute,aei.org,american enterprise institute
4,miot.mit.edu,Massachusetts Institute of Technology,mit.edu,massachusetts institute technology
...,...,...,...,...
1387,smsu.semo.edu,Southeast Missouri State University,semo.edu,southeast missouri state
1388,sc.statcan.gc.ca,Statistics Canada,statcan.gc.ca,statistics canada
1389,uofh.ufh.ac.za,University of Fort Hare,ufh.ac.za,fort hare
1390,uoi.unilorin.edu.ng,University of Ilorin,unilorin.edu.ng,ilorin


In [46]:
# university_df["institution"] = university_df["University"]

In [47]:
university_df["institution"] = university_df["University"].apply(lambda x: x.lower()).apply(lambda x: x.replace("university","")).apply(lambda x: x.replace("of",""))
university_df

,Unnamed: 0,University,Rank,institution
0,0,London School of Economics,1,london school economics
1,1,World Bank Group,2,world bank group
2,2,National Bureau of Economic Research,3,national bureau economic research
3,3,International Monetary Fund,4,international monetary fund
4,4,Harvard University,5,harvard
...,...,...,...,...
369,541,Bilkent Üniversitesi,542,bilkent üniversitesi
370,543,Royal Holloway,544,royal holloway
371,544,briq Institute on Behavior and Inequality,545,briq institute on behavior and inequality
372,546,Universitá della Svizzera Italiana,547,universitá della svizzera italiana


In [48]:
# Replace out the () to lower the possible noise
university_df["institution"] = university_df["institution"].apply(lambda x: re.sub("\(.*\)","",x))

In [49]:
university_df

,Unnamed: 0,University,Rank,institution
0,0,London School of Economics,1,london school economics
1,1,World Bank Group,2,world bank group
2,2,National Bureau of Economic Research,3,national bureau economic research
3,3,International Monetary Fund,4,international monetary fund
4,4,Harvard University,5,harvard
...,...,...,...,...
369,541,Bilkent Üniversitesi,542,bilkent üniversitesi
370,543,Royal Holloway,544,royal holloway
371,544,briq Institute on Behavior and Inequality,545,briq institute on behavior and inequality
372,546,Universitá della Svizzera Italiana,547,universitá della svizzera italiana


In [50]:
university_df = university_df.drop_duplicates()
university_df

,Unnamed: 0,University,Rank,institution
0,0,London School of Economics,1,london school economics
1,1,World Bank Group,2,world bank group
2,2,National Bureau of Economic Research,3,national bureau economic research
3,3,International Monetary Fund,4,international monetary fund
4,4,Harvard University,5,harvard
...,...,...,...,...
369,541,Bilkent Üniversitesi,542,bilkent üniversitesi
370,543,Royal Holloway,544,royal holloway
371,544,briq Institute on Behavior and Inequality,545,briq institute on behavior and inequality
372,546,Universitá della Svizzera Italiana,547,universitá della svizzera italiana


### INDEXING

In [51]:
index_full = Full()
candidate_links = index_full.index(authors_df, university_df)

our_comparison = recordlinkage.Compare()
# compare all our columns
our_comparison.string("institution", "institution", method = "jarowinkler", label = "jw_uni_name")
features = our_comparison.compute(candidate_links, authors_df, university_df)
features.head(10)


jw_uni_name
0 0     0.541667
  1     0.555556
  2     0.614379
  3     0.629630
  4     0.625000
  5     0.452381
  6     0.452381
  7     0.593651
  8     0.000000
  9     0.547619

In [52]:
features['jw_uni_name'].describe()

count    520608.000000
mean          0.502092
std           0.095609
min           0.000000
25%           0.454167
50%           0.506944
75%           0.557241
max           1.000000
Name: jw_uni_name, dtype: float64

In [53]:
thr_matches = features[(features["jw_uni_name"] >= 0)]
print("Threshold-based approach: {} matches".format(len(thr_matches)))
thr_matches

Threshold-based approach: 520608 matches


jw_uni_name
0    0       0.541667
     1       0.555556
     2       0.614379
     3       0.629630
     4       0.625000
...               ...
1391 369     0.366667
     370     0.465079
     371     0.561440
     372     0.584874
     373     0.499328

[520608 rows x 1 columns]

In [54]:
university_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374 entries, 0 to 373
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   374 non-null    int64 
 1   University   374 non-null    object
 2   Rank         374 non-null    int64 
 3   institution  374 non-null    object
dtypes: int64(2), object(2)
memory usage: 22.7+ KB


In [55]:
jwdic = {}
for i, row in thr_matches.iterrows():
    author_index = i[0]
    if author_index in jwdic:
        jwdic[author_index].append((row.jw_uni_name, i[1]))                                     
    else: 
        jwdic[author_index] = []
    

In [56]:
links = {"author_index":[], "university_index":[], "match_score":[]}
for key, values in jwdic.items():
    max_score = 0
    for value in values:
        if value[0] > max_score:
            max_score = value[0]
            index = value[1]
        else:
            continue
    links["author_index"].append(key)
    links["university_index"].append(index)
    links["match_score"].append(max_score)

In [57]:
links_df = pd.DataFrame(links)

In [58]:
links_df["author_ins"] = links_df["author_index"]
links_df["univer_ins"] = links_df["author_index"]
links_df["Rank"] = links_df["author_index"]

In [59]:
for r, row in links_df.iterrows():
    author_uni = authors_df.iloc[int(row.author_index)].institution
    univer = university_df.iloc[int(row.university_index)].University
    rank = university_df.iloc[int(row.university_index)].Rank
    links_df["author_ins"].at[r] = author_uni
    links_df["univer_ins"].at[r] = univer
    links_df["Rank"].at[r] = rank
links_df

,author_index,university_index,match_score,author_ins,univer_ins,Rank
0,0,227,0.796296,aalto,Carleton University,309
1,1,60,0.747008,kth royal institute technology,Beijing Institute of Technology,78
2,2,77,0.682900,los angeles,Boston College,100
3,3,160,0.759988,american enterprise institute,Economic and Social Research Institute,212
4,4,7,1.000000,massachusetts institute technology,Massachusetts Institute of Technology,8
...,...,...,...,...,...,...
1387,1387,299,0.824561,southeast missouri state,Southern Methodist University,423
1388,1388,247,0.686379,statistics canada,Oesterreichische Nationalbank,337
1389,1389,52,0.771717,fort hare,University of Notre Dame,69
1390,1390,138,0.826667,ilorin,University of Virginia,182


In [60]:
authors_df

,affiliationid,name,email,institution
0,au.aalto.fi,Aalto University,aalto.fi,aalto
1,kriot.kth.se,KTH Royal Institute of Technology,kth.se,kth royal institute technology
2,uocla.ucla.edu,"University of California, Los Angeles",ucla.edu,los angeles
3,aei.aei.org,American Enterprise Institute,aei.org,american enterprise institute
4,miot.mit.edu,Massachusetts Institute of Technology,mit.edu,massachusetts institute technology
...,...,...,...,...
1387,smsu.semo.edu,Southeast Missouri State University,semo.edu,southeast missouri state
1388,sc.statcan.gc.ca,Statistics Canada,statcan.gc.ca,statistics canada
1389,uofh.ufh.ac.za,University of Fort Hare,ufh.ac.za,fort hare
1390,uoi.unilorin.edu.ng,University of Ilorin,unilorin.edu.ng,ilorin


In [61]:
pd_con = pd.concat([authors_df, links_df], axis=1)
pd_con

,affiliationid,name,email,institution,author_index,university_index,match_score,author_ins,univer_ins,Rank
0,au.aalto.fi,Aalto University,aalto.fi,aalto,0,227,0.796296,aalto,Carleton University,309
1,kriot.kth.se,KTH Royal Institute of Technology,kth.se,kth royal institute technology,1,60,0.747008,kth royal institute technology,Beijing Institute of Technology,78
2,uocla.ucla.edu,"University of California, Los Angeles",ucla.edu,los angeles,2,77,0.682900,los angeles,Boston College,100
3,aei.aei.org,American Enterprise Institute,aei.org,american enterprise institute,3,160,0.759988,american enterprise institute,Economic and Social Research Institute,212
4,miot.mit.edu,Massachusetts Institute of Technology,mit.edu,massachusetts institute technology,4,7,1.000000,massachusetts institute technology,Massachusetts Institute of Technology,8
...,...,...,...,...,...,...,...,...,...,...
1387,smsu.semo.edu,Southeast Missouri State University,semo.edu,southeast missouri state,1387,299,0.824561,southeast missouri state,Southern Methodist University,423
1388,sc.statcan.gc.ca,Statistics Canada,statcan.gc.ca,statistics canada,1388,247,0.686379,statistics canada,Oesterreichische Nationalbank,337
1389,uofh.ufh.ac.za,University of Fort Hare,ufh.ac.za,fort hare,1389,52,0.771717,fort hare,University of Notre Dame,69
1390,uoi.unilorin.edu.ng,University of Ilorin,unilorin.edu.ng,ilorin,1390,138,0.826667,ilorin,University of Virginia,182


In [62]:
pd_con.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1392 entries, 0 to 1391
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   affiliationid     1392 non-null   object 
 1   name              1392 non-null   object 
 2   email             1392 non-null   object 
 3   institution       1392 non-null   object 
 4   author_index      1392 non-null   int64  
 5   university_index  1392 non-null   int64  
 6   match_score       1392 non-null   float64
 7   author_ins        1392 non-null   object 
 8   univer_ins        1392 non-null   object 
 9   Rank              1392 non-null   int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 108.9+ KB


In [63]:
affilication_ranking_linked = pd_con[["affiliationid", "name", "email", "match_score", "univer_ins", "Rank"]]

In [64]:
affilication_ranking_linked.to_csv("real_full_links.csv",index=False, encoding = "utf-8-sig")

In [69]:
len(affilication_ranking_linked[affilication_ranking_linked["match_score"]==1])

175

In [66]:
len(affilication_ranking_linked[affilication_ranking_linked["match_score"]>0.85])

395

In [ ]:
df=pd.read_csv("real_full_links.csv")